# **Tworzenie API we Flasku – Wprowadzenie**

W tym ćwiczeniu nauczysz się, jak stworzyć proste API w Flasku, uruchomić je, wysyłać do niego zapytania oraz wykorzystać model decyzyjny w oparciu o podstawową regułę logiczną.

## **1️⃣ Tworzenie podstawowego API**
Najpierw utworzymy podstawową aplikację Flask.

### **Zapisanie kodu API do pliku**
W Jupyter Notebooku użyj magicznej komendy `%%file`, aby zapisać kod podstawowej aplikacji flask do pliku `app.py`: Kod znajdziesz na [cw1](https://sebkaz-teaching.github.io/RTA_2025/cw1.html)
Jako tekst do wyświetlenie strony głównej użyj `Witaj w moim API!`.

In [ ]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

if __name__ == '__main__':
    app.run()

Writing app.py


Teraz uruchom API w terminalu, wpisując:
```sh
python app.py
```
Flask uruchomi serwer lokalnie pod adresem `http://127.0.0.1:5000/`.

### **Sprawdzenie działania API**
W Jupyter Notebooku wykonaj zapytanie GET do strony głównej. Na podstawie pola `status_code` napisz wyrażenie warunkowe które dla status_code 200 wyświetli zawartość odpowiedzi (z pola `content`).

In [8]:
import requests

response = requests.get("http://127.0.0.1:5000/")
if response.status_code == 200:
    print(response.content.decode())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ebca2d02c90>: Failed to establish a new connection: [Errno 111] Connection refused'))

Jeśli wszystko działa poprawnie, zobaczysz komunikat `Witaj w moim API!`.
---

## **2️⃣ Dodanie nowej podstrony**
Dodajmy nową podstronę `mojastrona`, która zwróci komunikat `To jest moja strona!`.

In [ ]:
%%file app.py
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return 'Witaj w moim API!'

@app.route('/mojastrona')
def mojastrona():
    return 'To jest moja strona!'

if __name__ == '__main__':
    app.run()


Overwriting app.py


Ponownie uruchom API i wykonaj zapytanie do strony `"http://127.0.0.1:5000/mojastrona"`:

In [ ]:
response = requests.get("http://127.0.0.1:5000/mojastrona")
if response.status_code == 200:
    print(response.content.decode())

SyntaxError: invalid syntax (<ipython-input-4-44f0a4939285>, line 1)

Powinieneś zobaczyć: `To jest moja strona!`

---

## **3️⃣ Automatyczne uruchamianie serwera z Jupyter Notebook**
Zamknij wcześniej uruchomiony serwer (`Ctrl+C` w terminalu) i uruchom go ponownie bezpośrednio z Jupyter Notebook, korzystając z `subprocess.Popen`:

In [9]:
import subprocess

server = subprocess.Popen(["python", "app.py"])

Po testach zamknij serwer wykorzystując metodę `kill`:


In [ ]:
server.kill()

---

## **4️⃣ Obsługa parametrów w adresie URL**
Dodajemy nową podstronę `/hello`, która będzie przyjmować parametr `name`.

Edytuj `app.py`, dodając odpowiedni  kod


In [ ]:
%%file app.py
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return 'Witaj w moim API!'

@app.route('/mojastrona')
def mojastrona():
    return 'To jest moja strona!'

@app.route('/hello')
def hello():
    name = request.args.get("name", "")
    if name:
        return f"Hello {name}!"
    else:
        return "Hello!"

if __name__ == '__main__':
    app.run()


In [14]:
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Krzysztof")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"

b'Hello!'
b'Hello Krzysztof!'


Uruchom serwer i sprawdź działanie API:
```python
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"
```

---

## **5️⃣ Tworzenie API z prostym modelem ML**
Stworzymy nową podstronę `/api/v1.0/predict`, która przyjmuje dwie liczby i zwraca wynik reguły decyzyjnej:
- Jeśli suma dwóch liczb jest większa niż 5.8, zwraca `1`.
- W przeciwnym razie zwraca `0`.


In [6]:
%%file app.py
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return 'Witaj w moim API!'

@app.route('/mojastrona')
def mojastrona():
    return 'To jest moja strona!'

@app.route('/hello')
def hello():
    name = request.args.get("name", "")
    return f"Hello {name}!" if name else "Hello!"

@app.route('/api/v1.0/predict')
def predict():
    try:
        num1 = float(request.args.get("num1", 0))
        num2 = float(request.args.get("num2", 0))
        result = 1 if (num1 + num2) > 5.8 else 0
        return jsonify({
            "prediction": result,
            "features": {
                "num1": num1,
                "num2": num2
            }
        })
    except:
        return jsonify({"error": "Błędne dane"}), 400

if __name__ == '__main__':
    app.run()


Overwriting app.py


In [12]:
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}

{'features': {'num1': 3.0, 'num2': 4.0}, 'prediction': 1}


Sprawdź działanie API:
```python
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}
```

---

## **Podsumowanie**
Po wykonaniu tego ćwiczenia studenci będą umieli:
✅ Tworzyć podstawowe API w Flasku.  
✅ Dodawać podstrony i obsługiwać parametry URL.  
✅ Wysyłać zapytania GET i analizować odpowiedzi.  
✅ Automatycznie uruchamiać serwer z Jupyter Notebook.  
✅ Implementować prosty model decyzyjny w API.  

Gotowi na kolejne wyzwania? 🚀